# 1. Import libraries

In [1]:
import pandas as pd
import numpy as np

import fastf1
from fastf1.ergast import Ergast

from tqdm import tqdm
import os

import sys
sys.path.append('../')

from src.race_prediction_model.extract import *
from src.race_prediction_model.feature_engineering import add_features_to_results

# Config less verbosity
fastf1.set_log_level('ERROR')

# 2. Data loading

Extraemos los circuitos (esto lo necesitamos antes)

In [2]:
# Poner esto también en A1-EDA-Clustering

try:
    df_races = pd.read_csv('../data/output/races.csv')

except FileNotFoundError:
    print("Data not found: Extracting data...")
    df_races = extract_races_dataframe(2010, end=2024, save=True)

In [3]:
df_races

,season,round,circuitId
0,2010,1,bahrain
1,2010,2,albert_park
2,2010,3,sepang
3,2010,4,shanghai
4,2010,5,catalunya
...,...,...,...
300,2024,20,rodriguez
301,2024,21,interlagos
302,2024,22,vegas
303,2024,23,losail


Tenemos 305 carreras desde 2010 hasta 2024. Extraemos los resultados

In [4]:
for season in df_races['season'].unique().tolist():

    print(season)

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [ ]:
season = 2024

results_final_df = extract_results_dataframe(df_races[df_races['season'] == season], save=False)[0]

results_final_df.to_csv(f'../data/output/results_{str(season)}.csv')

In [19]:
results_final_df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId
77,bottas,mercedes,1.0,2.0,0.000,Finished,26.0,2019,1,albert_park
44,hamilton,mercedes,2.0,1.0,20.886,Finished,18.0,2019,1,albert_park
33,max_verstappen,red_bull,3.0,4.0,22.520,Finished,15.0,2019,1,albert_park
5,vettel,ferrari,4.0,3.0,57.109,Finished,12.0,2019,1,albert_park
16,leclerc,ferrari,5.0,5.0,58.203,Finished,10.0,2019,1,albert_park
...,...,...,...,...,...,...,...,...,...,...
99,giovinazzi,alfa,16.0,16.0,169.205,+1 Lap,0.0,2019,21,yas_marina
63,russell,williams,17.0,18.0,169.205,+1 Lap,0.0,2019,21,yas_marina
10,gasly,toro_rosso,18.0,11.0,169.205,+2 Laps,0.0,2019,21,yas_marina
88,kubica,williams,19.0,19.0,169.205,+2 Laps,0.0,2019,21,yas_marina
